# Protein MD Predictor (v1) – Run on Google Colab

This notebook launches the Streamlit app inside Colab and exposes it with a temporary public URL.

**Notes**
- Colab sessions are temporary. If the runtime resets, rerun the cells.
- The public URL is reachable by anyone who has the link. Avoid sensitive data.
- For very large trajectories, Colab may be slow or memory-limited.


## 1) Get the code

Option A (recommended): clone from GitHub.

If you uploaded a ZIP instead, unzip it and `cd` into the project folder.


In [ ]:
!git clone https://github.com/sciencemaths-collab/protein-md-predictor.git
%cd protein-md-predictor


## 2) Install dependencies


In [ ]:
!pip -q install -r requirements.txt


## 3) Download Cloudflared (for a public URL)

Cloudflared creates a temporary `trycloudflare.com` URL without requiring accounts or tokens.


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


## 4) Launch Streamlit + Tunnel

Run this cell, then open the printed **Public URL**.


In [ ]:
import subprocess, time, re

# Start Streamlit (background)
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

# Give Streamlit a moment to boot
time.sleep(3)

# Start a Cloudflare tunnel (temporary public URL)
tunnel_proc = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8501", "--no-autoupdate"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

public_url = None
t0 = time.time()
while time.time() - t0 < 120:  # up to ~2 minutes
    line = tunnel_proc.stdout.readline()
    if not line:
        # If the process died, stop waiting
        if tunnel_proc.poll() is not None:
            break
        time.sleep(0.5)
        continue
    m = re.search(r"https://[\w-]+\.trycloudflare\.com", line)
    if m:
        public_url = m.group(0)
        break

print("\n✅ Streamlit is running on http://localhost:8501")
if public_url:
    print("🌐 Temporary public URL:", public_url)
    print("   Keep this Colab runtime running. The URL stops working when the tunnel stops.")
else:
    print("⚠️ Could not detect the public URL. Scroll cell output for a trycloudflare link.")

## Keep the link alive (optional)

If you want the temporary public URL to stay reachable, keep the tunnel running. This cell simply keeps the notebook active.

In [ ]:
import time
while True:
    time.sleep(60)

## 5) Stop the app (optional)
If you want to stop everything, run the cell below.


In [ ]:
import os, signal
for p in [globals().get('streamlit_proc'), globals().get('tunnel_proc')]:
    if p and p.poll() is None:
        p.send_signal(signal.SIGTERM)
print('Stopped (if running).')
